In [2]:
library(dplyr)
library(stringr)
library(readr)
library(tidyr)
library(lfe)
library(glmnet)
library(aod)
library(xgboost)
library(doMC)
registerDoMC(20)
source('~/covid_mobility/paper_code/utils/residualizer+genAlters.R')

In [3]:
load('/nfs/sloanlab004/projects/covid_mobility_proj/data/PROCESSED_DATA/data_pre_residualization.RData')

In [4]:
panel %>%
    mutate(ash_alter_rnstu   = asinh(alter_rnstu),
           ash_alter_nhd     = asinh(alter_nhd),
           log_alter_mcbgv   = asinh(alter_mcbgv)) -> panel

In [5]:
panel %>% 
    select(-matches('prcp..'), 
           -matches('tmax..'),
           -matches('_Xsh'), 
           -matches('_Xbg'),
           -device_count,
           -non_home_cbg_visits_within_county,
           -cbg_visits_outside_county,
           -home_cbg_visits,
           -outside_device_county_visits, 
           -ban_gatherings) -> panel

In [6]:
set.seed(2345)
panel %>%
    ungroup() %>%
    select(county_fips) %>%
    distinct() %>%
    mutate(i = sample(1:n(), n(), replace = F),
            grp = i %% 3 + 1) %>%
    select(-i) -> groups

panel %>%
    left_join(groups) -> panel

Joining, by = "county_fips"



In [7]:
gen_helper <- function(newPRCP, newTMAX) {
    panel %>%
        mutate(PRCP = newPRCP,
               TMAX = newTMAX,
               PRCP.fe.r = felm(PRCP ~ 0 | county_fips + ds)$resid, 
               TMAX.fe.r = felm(TMAX ~ 0 | county_fips + ds)$resid) -> FRI_df

    FRI_df %>%
        select(ds, county_fips, PRCP.fe.r, TMAX.fe.r, n) %>%
        left_join(groups) -> helper_frame
    return(helper_frame)
}

In [8]:
FRIresidualizer <- function(Y, colname, helper) {
    helper %>%
        mutate(Y = Y,
               Y.r = felm(Y ~ 0 | county_fips + ds, .)$resid) -> temp_df
    
    dm    <- xgb.DMatrix(data = model.matrix(~ 0 + PRCP.fe.r + TMAX.fe.r, temp_df), label = temp_df$Y.r)
    param <- list(max_depth=2, eta=.5, silent=1, objective='reg:linear')
    fit   <- xgb.cv(params = param, 
                    data = dm, 
                    folds = folds,
                    nrounds = 100, 
                    early_stopping_rounds = 3, 
                    weight = temp_df$n)
    best_n <- fit$best_iteration
    for (i in 1:3) {
        tr  <- temp_df %>% filter(grp != i)
        trm <- xgb.DMatrix(data = model.matrix(~ 0 + PRCP.fe.r + TMAX.fe.r, tr), label = tr$Y.r)
        fit <- xgb.train(params = param, 
                         data = trm, 
                         nrounds = best_n, 
                         weight = tr$n)
        te  <- temp_df %>% filter(grp == i)
        tem <- xgb.DMatrix(data = model.matrix(~ 0 + PRCP.fe.r + TMAX.fe.r, te), label = te$Y.r)
        te %>%
            select(county_fips, ds) %>%
            mutate(pred = predict(fit, newdata = tem)) -> pred_df
        assign(str_c('temp',i), pred_df)
    }
    out <- bind_rows(temp1, temp2, temp3) %>%
        arrange(ds, county_fips) %>%
        mutate(tempname = temp_df$Y.r - pred) %>%
        select(-pred, -ds, -county_fips)
    colnames(out) <- str_c(colname, '.r')
    return(out)
}

In [9]:
gen_FRI_data <- function(newPRCP, newTMAX) {
    panel %>%
        mutate(PRCP = newPRCP,
               TMAX = newTMAX,
               PRCP.fe.r = felm(PRCP ~ 0 | county_fips + ds)$resid, 
               TMAX.fe.r = felm(TMAX ~ 0 | county_fips + ds)$resid) %>%
        mutate(rain = as.numeric(PRCP > 0)) %>%     ## feel free to add instruments here
        mutate(alter_rain = weightedAlters(., WM, rain)) -> FRI_df
    ## due to how weightedAlters works, it cannot be in same mutate that the base variable was created in

    FRI_df %>%
        select(ds, county_fips, PRCP.fe.r, TMAX.fe.r, n, grp) %>%
        left_join(groups) -> helper_frame
    
    FRI_df %>% 
        select(-ds, 
               -county_fips, 
               -state_abbv, 
               -n, 
               -PRCP, 
               -TMAX, 
               -PRCP.fe.r, 
               -TMAX.fe.r, 
               -grp) %>%
        colnames() -> cols_to_xgr
    print(cols_to_xgr)
    
    xg.residuals <- foreach(i = 1:length(cols_to_xgr), .combine = cbind) %do% 
        FRIresidualizer(FRI_df[[cols_to_xgr[i]]], cols_to_xgr[i], helper_frame)
    FRI_df %>%
        bind_cols(xg.residuals) -> FRI_df
    return(FRI_df)   
}

In [10]:
### Generate New FRI-ed Df
### inputs are vectors that require the observations be in the following order:
### date1, county1, ...
### date1, county2, ...
### ...
### date2, county1, ...
### date2, county2, ...
FRI <- gen_FRI_data(panel$PRCP, panel$TMAX)

Joining, by = c("county_fips", "grp")



 [1] "sg_nhd"          "stay_home"       "n11"             "n21"            
 [5] "n22"             "n23"             "n31"             "n32"            
 [9] "n33"             "n42"             "n44"             "n45"            
[13] "n48"             "n49"             "n51"             "n52"            
[17] "n53"             "n54"             "n55"             "n56"            
[21] "n61"             "n62"             "n71"             "n72"            
[25] "n81"             "n92"             "nNA"             "fb_btvrc"       
[29] "fb_rnstu"        "sg_mcbgv"        "ban_gmr"         "log_mcbgv"      
[33] "ash_nhd"         "ash_rnstu"       "alter_n11"       "alter_n21"      
[37] "alter_n22"       "alter_n23"       "alter_n31"       "alter_n32"      
[41] "alter_n33"       "alter_n42"       "alter_n44"       "alter_n45"      
[45] "alter_n48"       "alter_n49"       "alter_n51"       "alter_n52"      
[49] "alter_n53"       "alter_n54"       "alter_n55"       "alter_n56"      

In [12]:
names(FRI)
dim(FRI)

[1] "ds"                "county_fips"       "sg_nhd"           
  [4] "state_abbv"        "stay_home"         "PRCP"             
  [7] "TMAX"              "n11"               "n21"              
 [10] "n22"               "n23"               "n31"              
 [13] "n32"               "n33"               "n42"              
 [16] "n44"               "n45"               "n48"              
 [19] "n49"               "n51"               "n52"              
 [22] "n53"               "n54"               "n55"              
 [25] "n56"               "n61"               "n62"              
 [28] "n71"               "n72"               "n81"              
 [31] "n92"               "nNA"               "fb_btvrc"         
 [34] "fb_rnstu"          "n"                 "sg_mcbgv"         
 [37] "ban_gmr"           "log_mcbgv"         "ash_nhd"          
 [40] "ash_rnstu"         "alter_n11"         "alter_n21"        
 [43] "alter_n22"         "alter_n23"         "alter_n31"        
 [46] "alter_n32"         "alter_n33"         "alter_n42"        
 [49] "alter_n44"         "alter_n45"         "alter_n48"        
 [52] "alter_n49"         "alter_n51"         "alter_n52"        
 [55] "alter_n53"         "alter_n54"         "alter_n55"        
 [58] "alter_n56"         "alter_n61"         "alter_n62"        
 [61] "alter_n71"         "alter_n72"         "alter_n81"        
 [64] "alter_n92"         "alter_log_mcbgv"   "alter_ash_nhd"    
 [67] "alter_ash_rnstu"   "alter_nhd"         "alter_mcbgv"      
 [70] "alter_rnstu"       "alter_btvrc"       "alter_sh"         
 [73] "alter_bg"          "alter_bgmr"        "geo_alter_sh"     
 [76] "geo_alter_bg"      "geo_alter_bgmr"    "ash_alter_rnstu"  
 [79] "ash_alter_nhd"     "log_alter_mcbgv"   "grp"              
 [82] "PRCP.fe.r"         "TMAX.fe.r"         "rain"             
 [85] "alter_rain"        "sg_nhd.r"          "stay_home.r"      
 [88] "n11.r"             "n21.r"             "n22.r"            
 [91] "n23.r"             "n31.r"             "n32.r"            
 [94] "n33.r"             "n42.r"             "n44.r"            
 [97] "n45.r"             "n48.r"             "n49.r"            
[100] "n51.r"             "n52.r"             "n53.r"            
[103] "n54.r"             "n55.r"             "n56.r"            
[106] "n61.r"             "n62.r"             "n71.r"            
[109] "n72.r"             "n81.r"             "n92.r"            
[112] "nNA.r"             "fb_btvrc.r"        "fb_rnstu.r"       
[115] "sg_mcbgv.r"        "ban_gmr.r"         "log_mcbgv.r"      
[118] "ash_nhd.r"         "ash_rnstu.r"       "alter_n11.r"      
[121] "alter_n21.r"       "alter_n22.r"       "alter_n23.r"      
[124] "alter_n31.r"       "alter_n32.r"       "alter_n33.r"      
[127] "alter_n42.r"       "alter_n44.r"       "alter_n45.r"      
[130] "alter_n48.r"       "alter_n49.r"       "alter_n51.r"      
[133] "alter_n52.r"       "alter_n53.r"       "alter_n54.r"      
[136] "alter_n55.r"       "alter_n56.r"       "alter_n61.r"      
[139] "alter_n62.r"       "alter_n71.r"       "alter_n72.r"      
[142] "alter_n81.r"       "alter_n92.r"       "alter_log_mcbgv.r"
[145] "alter_ash_nhd.r"   "alter_ash_rnstu.r" "alter_nhd.r"      
[148] "alter_mcbgv.r"     "alter_rnstu.r"     "alter_btvrc.r"    
[151] "alter_sh.r"        "alter_bg.r"        "alter_bgmr.r"     
[154] "geo_alter_sh.r"    "geo_alter_bg.r"    "geo_alter_bgmr.r" 
[157] "ash_alter_rnstu.r" "ash_alter_nhd.r"   "log_alter_mcbgv.r"
[160] "rain.r"            "alter_rain.r"

[1] 122598    161

In [14]:
### If you need to add a column to the FRI_df after the fact:
### Using current PRCP and TMAX as an example
hf <- gen_helper(panel$PRCP, panel$TMAX)
FRI %>%
    mutate(rain = as.numeric(PRCP > 0)) %>%
    mutate(alter_rain = weightedAlters(., WM, rain)) -> FRI
FRI %>%
    bind_cols(FRIresidualizer(FRI$rain, 'rain', hf), 
              FRIresidualizer(FRI$alter_rain, 'alter_rain', hf)) -> FRI


Joining, by = "county_fips"



In [31]:
### Simple model
fit <- felm(fb_btvrc.r ~ 0 + ban_gmr.r + geo_alter_bgmr.r + alter_bgmr.r + 
            stay_home.r + geo_alter_sh.r + alter_sh.r | 
            0 | 
            (alter_btvrc.r ~ alter_rain.r) | 0, FRI)
summary(fit)


Call:
   felm(formula = fb_btvrc.r ~ 0 + ban_gmr.r + geo_alter_bgmr.r +      alter_bgmr.r + stay_home.r + geo_alter_sh.r + alter_sh.r |      0 | (alter_btvrc.r ~ alter_rain.r) | 0, data = FRI) 

Residuals:
     Min       1Q   Median       3Q      Max 
-0.38833 -0.01939 -0.00087  0.01815  0.42961 

Coefficients:
                      Estimate Std. Error t value Pr(>|t|)    
ban_gmr.r            -0.007534   0.001512  -4.984 6.24e-07 ***
geo_alter_bgmr.r     -0.002024   0.002449  -0.826 0.408628    
alter_bgmr.r          0.014928   0.003898   3.829 0.000128 ***
stay_home.r          -0.027327   0.001843 -14.828  < 2e-16 ***
geo_alter_sh.r       -0.003683   0.002878  -1.280 0.200679    
alter_sh.r            0.042538   0.004987   8.530  < 2e-16 ***
`alter_btvrc.r(fit)`  1.497246   0.014776 101.328  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.03334 on 122591 degrees of freedom
Multiple R-squared(full model): 0.569   Adjusted R-s